##### kernel analysis

In [1]:
import numpy as np
import pickle
from src.load_results import *
from src.kernels_list import *

##### printing results in a txt file

In [2]:
ajout = '' ## '' or '_clusterCV' 


if ajout=='':
    list_mol_Kernel = make_mol_kernel_list()
    list_prot_Kernel = make_prot_kernel_list()
elif ajout=='_clusterCV':
    list_mol_Kernel = make_mol_kernel_list_ClusterCV()
    list_prot_Kernel = make_prot_kernel_list_ClusterCV()

dico_result_per_kernel = {}
dico_stdev_result_per_kernel = {}
dico_result_association = {}
list_param = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]
nb_average =  4
nb_fold = 5

ind_mol_kernel = list_mol_Kernel.index('Tanimoto_d=8')
ind_prot_kernel = list_prot_Kernel.index('LAkernel_beta_0.5_g_100_0.01_ln_normed')
print(ind_mol_kernel)
print(ind_prot_kernel)

list_best_score = []
f_out = open("saved_results/ChoiceKernel"+ajout+"_result/global.txt", 'w')
for indice_1 in range(len(list_mol_Kernel)):
    print(list_mol_Kernel[indice_1])
    if list_mol_Kernel[indice_1] not in dico_result_per_kernel.keys():
        dico_result_per_kernel[list_mol_Kernel[indice_1]] = []
        dico_stdev_result_per_kernel[list_mol_Kernel[indice_1]] = []
        dico_result_association[list_mol_Kernel[indice_1]] = []
    for indice_2 in range(len(list_prot_Kernel)):
            if list_prot_Kernel[indice_2] not in dico_result_per_kernel.keys():
                dico_result_per_kernel[list_prot_Kernel[indice_2]] = []
                dico_stdev_result_per_kernel[list_prot_Kernel[indice_2]] = []
                dico_result_association[list_prot_Kernel[indice_2]] = []

            list_mean_auc_depending_on_PartialNeg_nested = pickle.load(open("saved_results/MT_KernelAnalysis/ChoiceKernel"+ajout+"_S1_SVM_auc-nested_"+str(indice_1)+"_"+str(indice_2)+".data", 'rb'))
            list_mean_aupr_depending_on_PartialNeg_nested = pickle.load(open("saved_results/MT_KernelAnalysis/ChoiceKernel"+ajout+"_S1_SVM_aupr-nested_"+str(indice_1)+"_"+str(indice_2)+".data", 'rb'))
            list_mean_auc_depending_on_PartialNeg = pickle.load(open("saved_results/MT_KernelAnalysis/ChoiceKernel"+ajout+"_S1_SVM_auc_"+str(indice_1)+"_"+str(indice_2)+".data", 'rb'))
            list_mean_aupr_depending_on_PartialNeg = pickle.load(open("saved_results/MT_KernelAnalysis/ChoiceKernel"+ajout+"_S1_SVM_aupr_"+str(indice_1)+"_"+str(indice_2)+".data", 'rb'))

            list_aupr_score = []
            list_auc_score = []
            for ind_partial_neg_couples in range(nb_average):
                list_best_aupr_on_NestedTest = []
                list_best_auc_on_NestedTest = []
                for ind_fold_nested in range(nb_fold):
                    list_aupr_avg = []
                    for ind_temp in range(len(list_param)):
                        list_aupr_avg.append(0)
                    for ind_fold in range(nb_fold-1):
                        for param in range(len(list_param)):
                            #print(len(list_mean_aupr_depending_on_PartialNeg[param][ind_partial_neg_couples]))
                            list_aupr_avg[param] += list_mean_aupr_depending_on_PartialNeg[param][ind_partial_neg_couples][ind_fold_nested*(nb_fold-1)+ind_fold]
                    for param in range(len(list_param)):
                        list_aupr_avg[param] = list_aupr_avg[param]/(nb_fold-1)
                        if indice_1==ind_mol_kernel and indice_2==ind_prot_kernel:
                            print("C="+str(list_param[param])+':'+str(list_aupr_avg[param]))
                    best_param = np.argmax(list_aupr_avg)
                    if indice_1==ind_mol_kernel and indice_2==ind_prot_kernel:
                        print("best_param=", list_param[best_param])

                    list_best_aupr_on_NestedTest.append(list_mean_aupr_depending_on_PartialNeg_nested[best_param][ind_partial_neg_couples][ind_fold_nested])
                    list_best_auc_on_NestedTest.append(list_mean_auc_depending_on_PartialNeg_nested[best_param][ind_partial_neg_couples][ind_fold_nested])
                list_aupr_score.append(statistics.mean(list_best_aupr_on_NestedTest))
                list_auc_score.append(statistics.mean(list_best_auc_on_NestedTest))
                
            #print(list_mol_Kernel[indice_1])
            #print(list_prot_Kernel[indice_2])
            f_out.write(list_mol_Kernel[indice_1]+'\t'+list_prot_Kernel[indice_2]+'\t'+str(statistics.mean(list_auc_score))+'\t'+str(statistics.stdev(list_auc_score))+'\t'+str(statistics.mean(list_aupr_score))+'\t'+str(statistics.stdev(list_aupr_score))+'\n')

            dico_result_per_kernel[list_mol_Kernel[indice_1]].append(statistics.mean(list_aupr_score))
            dico_stdev_result_per_kernel[list_mol_Kernel[indice_1]].append(statistics.stdev(list_aupr_score))
            dico_result_association[list_mol_Kernel[indice_1]].append(list_prot_Kernel[indice_2])
            dico_result_per_kernel[list_prot_Kernel[indice_2]].append(statistics.mean(list_aupr_score))
            dico_stdev_result_per_kernel[list_prot_Kernel[indice_2]].append(statistics.stdev(list_aupr_score))
            dico_result_association[list_prot_Kernel[indice_2]].append(list_mol_Kernel[indice_1])

            list_best_score.append(statistics.mean(list_aupr_score))
        #else:
            #list_best_score.append(0)
best = np.argmax(list_best_score)
f_out.write("####################")
print(len(list_mol_Kernel))
print(len(list_prot_Kernel))
print(best//len(list_mol_Kernel))
print(best%len(list_mol_Kernel))
f_out.write(list_mol_Kernel[best//len(list_prot_Kernel)]+'\t'+list_prot_Kernel[best%len(list_prot_Kernel)]+'\t'+str(list_best_score[best])+'\n')
f_out.close()


dico_best_result = {'SW':[],'LAkernel':[], 'ProfileKernel':[], 'RandomWalk':[], 'Tanimoto':[], 'Tanimoto_MinMax':[]}
dico_best_result_stdev = {'SW':[],'LAkernel':[], 'ProfileKernel':[], 'RandomWalk':[], 'Tanimoto':[], 'Tanimoto_MinMax':[]}
dico_best_result_association = {'SW':[],'LAkernel':[], 'ProfileKernel':[], 'RandomWalk':[], 'Tanimoto':[], 'Tanimoto_MinMax':[]}

for mol_kernel in list_mol_Kernel:
    f_out = open('saved_results/ChoiceKernel'+ajout+'_result/'+mol_kernel+'.txt', 'w')
    for ind, prot_kernel in enumerate(dico_result_association[mol_kernel]):
        f_out.write(prot_kernel+'\t'+str(dico_result_per_kernel[mol_kernel][ind])+'\t'+str(dico_stdev_result_per_kernel[mol_kernel][ind])+'\n')
    f_out.write('###########\n')
    for _ in range(5):
        ind_max = np.argmax(dico_result_per_kernel[mol_kernel])
        f_out.write(dico_result_association[mol_kernel][ind_max]+'\t'+str(dico_result_per_kernel[mol_kernel][ind_max])+'\t'+str(dico_stdev_result_per_kernel[mol_kernel][ind_max])+'\n')

        if mol_kernel[:10]=='RandomWalk':
            #print("RandomWalk")
            dico_best_result['RandomWalk'].append(dico_result_per_kernel[mol_kernel][ind_max])
            dico_best_result_association['RandomWalk'].append(mol_kernel+'\t'+dico_result_association[mol_kernel][ind_max])
            dico_best_result_stdev['RandomWalk'].append(dico_stdev_result_per_kernel[mol_kernel][ind_max])
        elif mol_kernel[:14]=='TanimotoMinMax':
            #print('TanimotoMinMax')
            dico_best_result['Tanimoto_MinMax'].append(dico_result_per_kernel[mol_kernel][ind_max])
            dico_best_result_association['Tanimoto_MinMax'].append(mol_kernel+'\t'+dico_result_association[mol_kernel][ind_max])
            dico_best_result_stdev['Tanimoto_MinMax'].append(dico_stdev_result_per_kernel[mol_kernel][ind_max])
        elif mol_kernel[:8]=='Tanimoto':
            #print('Tanimoto')
            dico_best_result['Tanimoto'].append(dico_result_per_kernel[mol_kernel][ind_max])
            dico_best_result_association['Tanimoto'].append(mol_kernel+'\t'+dico_result_association[mol_kernel][ind_max])
            dico_best_result_stdev['Tanimoto'].append(dico_stdev_result_per_kernel[mol_kernel][ind_max])
        else:
            print(mol_kernel[:14])

        del dico_result_association[mol_kernel][ind_max]
        del dico_result_per_kernel[mol_kernel][ind_max]
        del dico_stdev_result_per_kernel[mol_kernel][ind_max]
    f_out.close()

for prot_kernel in list_prot_Kernel:
    f_out = open('saved_results/ChoiceKernel'+ajout+'_result/'+prot_kernel+'.txt', 'w')
    for ind, mol_kernel in enumerate(dico_result_association[prot_kernel]):
        f_out.write(mol_kernel+'\t'+str(dico_result_per_kernel[prot_kernel][ind])+'\t'+str(dico_stdev_result_per_kernel[prot_kernel][ind])+'\n')
    f_out.write('############\n')
    for _ in range(5):
        ind_max = np.argmax(dico_result_per_kernel[prot_kernel])
        f_out.write(dico_result_association[prot_kernel][ind_max]+'\t'+str(dico_result_per_kernel[prot_kernel][ind_max])+'\t'+str(dico_stdev_result_per_kernel[prot_kernel][ind_max])+'\n')

        if prot_kernel[:8]=='LAkernel':
            #print('LAkernel')
            dico_best_result['LAkernel'].append(dico_result_per_kernel[prot_kernel][ind_max])
            dico_best_result_association['LAkernel'].append(prot_kernel+'\t'+dico_result_association[prot_kernel][ind_max])
            dico_best_result_stdev['LAkernel'].append(dico_stdev_result_per_kernel[prot_kernel][ind_max])
        elif prot_kernel[:30]=='allDrugBankHumanTarget_Profile':
            #print('Profile')
            dico_best_result['ProfileKernel'].append(dico_result_per_kernel[prot_kernel][ind_max])
            dico_best_result_association['ProfileKernel'].append(prot_kernel+'\t'+dico_result_association[prot_kernel][ind_max])
            dico_best_result_stdev['ProfileKernel'].append(dico_stdev_result_per_kernel[prot_kernel][ind_max])
        elif prot_kernel[:8]=='KernelSW':
            #print('LAkernel')
            dico_best_result['SW'].append(dico_result_per_kernel[prot_kernel][ind_max])
            dico_best_result_association['SW'].append(prot_kernel+'\t'+dico_result_association[prot_kernel][ind_max])
            dico_best_result_stdev['SW'].append(dico_stdev_result_per_kernel[prot_kernel][ind_max])

        del dico_result_association[prot_kernel][ind_max]
        del dico_result_per_kernel[prot_kernel][ind_max]
        del dico_stdev_result_per_kernel[prot_kernel][ind_max]
    f_out.close()


for kernel in dico_best_result.keys():
    f_out = open('saved_results/ChoiceKernel'+ajout+'_result/'+kernel+'.txt', 'w')
    for ind, value in enumerate(dico_best_result_association[kernel]):
        f_out.write(value+'\t'+str(dico_best_result[kernel][ind])+'\n')
    f_out.write('#############\n')
    #for _ in range(20):
     #   ind_max = np.argmax(dico_best_result[kernel])
     #   f_out.write(dico_best_result_association[kernel][ind_max]+'\t'+str(dico_best_result[kernel][ind_max])+'\t'+str(dico_best_result_stdev[kernel][ind_max])+'\n')
     #   del dico_best_result[kernel][ind_max]
     #   del dico_best_result_association[kernel][ind_max]
     #   del dico_best_result_stdev[kernel][ind_max]
    f_out.close()

len(list_mol_kernel_name)= 24
len(list_prot_kernel_name)= 105
14
76
RandomWalkBasedKernel_q=0.05_MI=2
RandomWalkBasedKernel_q=0.05_MI=4
RandomWalkBasedKernel_q=0.05_MI=6
RandomWalkBasedKernel_q=0.1_MI=2
RandomWalkBasedKernel_q=0.1_MI=4
RandomWalkBasedKernel_q=0.1_MI=6
RandomWalkBasedKernel_q=0.5_MI=2
RandomWalkBasedKernel_q=0.5_MI=4
RandomWalkBasedKernel_q=0.5_MI=6
TanimotoMinMax_d=2
TanimotoMinMax_d=4
TanimotoMinMax_d=6
TanimotoMinMax_d=8
Tanimoto_d=6
Tanimoto_d=8
C=1e-05:0.888777303052
C=0.0001:0.888772253709
C=0.001:0.888771906174
C=0.01:0.888771882422
C=0.1:0.893354935252
C=1:0.926076751182
C=10:0.928334112314
C=100:0.928323788238
C=1000:0.928323788238
C=10000:0.928323788238
C=100000:0.928323788238
best_param= 10
C=1e-05:0.888666233051
C=0.0001:0.888655874463
C=0.001:0.888655267888
C=0.01:0.888655547139
C=0.1:0.892955461179
C=1:0.926077743981
C=10:0.928687577623
C=100:0.928690323102
C=1000:0.928690323102
C=10000:0.928690323102
C=100000:0.928690323102
best_param= 100
C=1e-05:0.89108